In [1]:
library(terra)
library(sf)
library(tidyterra)
library(MultiscaleDTM)
library(randomForest)
library(caret)
library(vip)
library(rstac)


s_obj <- stac("https://planetarycomputer.microsoft.com/api/stac/v1")

terra 1.7.71

Linking to GEOS 3.10.2, GDAL 3.4.3, PROJ 8.2.0; sf_use_s2() is TRUE


Attaching package: ‘tidyterra’


The following object is masked from ‘package:stats’:

    filter


Warning message in rgl.init(initValue, onlyNULL):
“RGL: unable to open X11 display”
Warning message:
“'rgl.init' failed, running with 'rgl.useNULL = TRUE'.”
randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


Loading required package: lattice


Attaching package: ‘vip’


The following object is masked from ‘package:utils’:

    vi




In [38]:
it_obj <- s_obj %>% 
    stac_search(collections = "sentinel-2-l2a",
                datetime = "2018-07-01T00:00:00Z/2018-08-30T00:00:00Z",
                limit = 10,
                bbox = c(-47.02148, -17.35063, -42.53906, -12.98314)) %>%
    get_request()

print(it_obj)

###STACItemCollection
- features (10 item(s)):
  - S2A_MSIL2A_20180828T132231_R038_T23LNF_20201027T054353
  - S2A_MSIL2A_20180828T132231_R038_T23LNE_20201027T054349
  - S2A_MSIL2A_20180828T132231_R038_T23LND_20201027T054349
  - S2A_MSIL2A_20180828T132231_R038_T23LMF_20201027T054359
  - S2A_MSIL2A_20180828T132231_R038_T23LME_20201027T054358
  - S2A_MSIL2A_20180828T132231_R038_T23LMD_20201027T054354
  - S2A_MSIL2A_20180828T132231_R038_T23LMC_20201027T054355
  - S2A_MSIL2A_20180828T132231_R038_T23LLF_20201027T054355
  - S2A_MSIL2A_20180828T132231_R038_T23LLE_20201027T054356
  - S2A_MSIL2A_20180828T132231_R038_T23LLD_20201027T054357
- assets: 
AOT, B01, B02, B03, B04, B05, B06, B07, B08, B09, B11, B12, B8A, datastrip-metadata, granule-metadata, inspire-metadata, preview, product-metadata, rendered_preview, safe-manifest, SCL, tilejson, visual, WVP
- item's fields: 
assets, bbox, collection, geometry, id, links, properties, stac_extensions, stac_version, type


In [45]:
rast(it_obj$features[[1]]$assets$B04$href)

Warning message in new_CppObject_xp(fields$.module, fields$.pointer, ...):
“GDAL Error 1: HTTP error code : 404”


ERROR: Error: [rast] file does not exist: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/23/L/NF/2018/08/28/S2A_MSIL2A_20180828T132231_N0212_R038_T23LNF_20201027T054353.SAFE/GRANULE/L2A_T23LNF_A016623_20180828T132231/IMG_DATA/R10m/T23LNF_20180828T132231_B04_10m.tif


In [40]:
?stac_image_collection


No documentation for ‘stac_image_collection’ in specified packages and libraries:
you could try ‘??stac_image_collection’